# Procesamiento de Lenguaje Natural - CEIA
# Desafío 1
## Alumno: Hernán Matías Silva

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


## 1

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import random

# Cargar los datos
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

# Instanciar y ajustar el vectorizador
tfidfvect = TfidfVectorizer(
    stop_words='english',   # Eliminar las palabras comunes del inglés
    max_df=0.8,             # Elminiar los términos que aparecen en más del 80% de los documentos
    min_df=3,               # Eliminar los términos que aparecen en menos de 3 documentos
    ngram_range=(1, 2),     # Considerar unigramas y bigramas 
    max_features=10000,     # Limitar el número de características a las 10000 más relevantes
    sublinear_tf=True       # Aplicar una escala sublineal a los términos
)
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

# Semilla 
random.seed(42)

# Seleccionar índices de 5 documentos al azar
cant_doc = X_train.shape[0]
doc_indices = random.sample(range(cant_doc), 5)

# Iterar sobre cada documento seleccionado
for idx in doc_indices:
    print("=" * 80)
    print(f"Documento original (Índice: {idx}):")
    print(newsgroups_train.data[idx][:500])  # Mostrar solo los primeros 500 caracteres
    print(f"\nClase: {newsgroups_train.target_names[y_train[idx]]}")
    print("=" * 80)
    
    # Calcular similitud coseno con todos los documentos
    cossim = cosine_similarity(X_train[idx], X_train)[0]
    
    # Obtener los índices de los 5 documentos más similares (excluyendo el propio documento)
    most_similar_indices = np.argsort(cossim)[::-1][1:6]
    
    print("Documentos más similares:\n")
    for sim_idx in most_similar_indices:
        print(f"- Documento (Índice: {sim_idx}):")
        print(newsgroups_train.data[sim_idx][:500])  # Mostrar solo los primeros 500 caracteres
        print(f"  Clase: {newsgroups_train.target_names[y_train[sim_idx]]}")
        print("-" * 40)


Documento original (Índice: 10476):
This is a general question for US readers:

How extensive is the playoff coverage down there?  In Canada, it is almost
impossible not to watch a series on TV (ie the only two series I have not had
an opportunity to watch this year are Wash-NYI and Chi-Stl, the latter because
I'm in the wrong time zone!).  We (in Canada) are basically swamped with 
coverage, and I wonder how many series/games are televised nationally or even
locally in the US and how much precedence they take over, say, local new

Clase: rec.sport.hockey
Documentos más similares:

- Documento (Índice: 5064):

I only have one comment on this:  You call this a *classic* playoff year
and yet you don't include a Chicago-Detroit series.  C'mon, I'm a Boston
fan and I even realize that Chicago-Detroit games are THE most exciting
games to watch.
  Clase: rec.sport.hockey
----------------------------------------
- Documento (Índice: 1988):
OK, I'll join in the fun and give my playoff predicti

1. Para el documento 10476 de la clase rec.sport.hockey que habla sobre la cobertura de los playoff, los primeros 4 documentos son de la misma clase que el original y hablan de los playoff, 3 de ellos de predicciones de juegos. El último documento a pesar de ser de otra clase rec.sport.baseball tiene similitud con el original ya que habla de la cobertura de un equipo. 
2. Para el documento 1824 de la clase comp.sys.mac.hardware que habla sobre la comparación del rendimiento de computadoras, los 5 documentos corresponden a la misma clase y los 3 primeros también hablan del mismo tema muy similares al original. Los últimos dos comentan sobre el hardware de computadoras.
3. Para el documento 409 de la clase comp.graphics que comenta sobre un problema con archivos .pov al compilar una aplicación, los primeros 4 documentos son de la misma clase y tres de ellos comentan de archivos el cuarto mensiona la sigla POV. El último documento  es de la clase comp.windows.x solo mensiona SunOS igual que el documento original.
4. Para el documento 4506 de la clase rec.autos que consulta sobre el cartel de aceite del motor. Hay 4 documentos de la misma clase que el original y otro de rec.motorcycles. Cuatro documentos hablan del aceite en el motor. Un documento sólo concuerda con el original en la palabra "sound", que es un verbo intransitivo en este caso, pero habla de un tema distinto.
5. Para el documento 4012 de la clase rec.sport.hockey que habla sobre jugadores de un equipo Leafs de hockey. Hay 5 documentos de la misma clase que el original, 4 nombran al equipo Leafs y uno a Winnipeg que también se nombra en el original.

# 2

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

# Cargar datos y dividir en train y test
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
X, y = newsgroups.data, newsgroups.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir rangos de hiperparámetros para el vectorizador y los modelos
vectorizer_params = {
    "max_df": [0.8, 0.9],
    "min_df": [3, 5],
    "ngram_range": [(1, 1), (1, 2)],
    "max_features": [5000, 10000, None],
    "sublinear_tf": [True, False],
    "stop_words": ['english']
}

model_params = {
    "alpha": [0.5, 1.0, 2.0]
}

best_f1_score = 0
best_params = {}

# Exploración de hiperparámetros
for max_df in vectorizer_params["max_df"]:
    for min_df in vectorizer_params["min_df"]:
        for ngram_range in vectorizer_params["ngram_range"]:
            for max_features in vectorizer_params["max_features"]:
                for sublinear_tf in vectorizer_params["sublinear_tf"]:
                    # Configurar y ajustar el vectorizador
                    tfidfvect = TfidfVectorizer(
                        stop_words='english',
                        max_df=max_df,
                        min_df=min_df,
                        ngram_range=ngram_range,
                        max_features=max_features,
                        sublinear_tf=sublinear_tf
                    )
                    X_train_vect = tfidfvect.fit_transform(X_train)
                    X_test_vect = tfidfvect.transform(X_test)

                    # Probar MultinomialNB y ComplementNB con diferentes alphas
                    for alpha in model_params["alpha"]:
                        for model_type in [MultinomialNB, ComplementNB]:
                            clf = model_type(alpha=alpha)
                            clf.fit(X_train_vect, y_train)
                            y_pred = clf.predict(X_test_vect)

                            # Calcular el F1-score macro
                            f1_macro = f1_score(y_test, y_pred, average='macro')

                            # Guardar mejores resultados
                            if f1_macro > best_f1_score:
                                best_f1_score = f1_macro
                                best_params = {
                                    "model": model_type.__name__,
                                    "alpha": alpha,
                                    "max_df": max_df,
                                    "min_df": min_df,
                                    "ngram_range": ngram_range,
                                    "max_features": max_features,
                                    "sublinear_tf": sublinear_tf
                                }

# Imprimir los mejores resultados
print("Mejor F1-score macro:", best_f1_score)
print("Mejores parámetros:", best_params)


Mejor F1-score macro: 0.7532505558566581
Mejores parámetros: {'model': 'ComplementNB', 'alpha': 0.5, 'max_df': 0.8, 'min_df': 3, 'ngram_range': (1, 2), 'max_features': None, 'sublinear_tf': True}


# 3

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Cargar los datos
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
X, y = newsgroups.data, newsgroups.target

# Configurar y ajustar el vectorizador
tfidfvect = TfidfVectorizer(
    stop_words='english',
    max_df=0.8,
    min_df=5,
    ngram_range=(1, 1),  # Unigramas para palabras individuales
    max_features=5000,
    sublinear_tf=True
)
X_tfidf = tfidfvect.fit_transform(X)

# Transponer la matriz documento-término
X_terms_docs = X_tfidf.T

# Obtener lista de términos
terms = tfidfvect.get_feature_names_out()

# Seleccionar palabras manualmente
selected_words = ["computer", "science", "software", "system", "data"]

# Iterar sobre las palabras seleccionadas
for word in selected_words:
    if word in terms:
        word_idx = np.where(terms == word)[0][0]
        print("=" * 80)
        print(f"Palabra: {word}")
        print("=" * 80)

        # Calcular similitud coseno con otras palabras
        word_vector = X_terms_docs[word_idx]
        cossim = cosine_similarity(word_vector, X_terms_docs)[0]

        # Obtener índices de las 5 palabras más similares (excluyendo la palabra misma)
        most_similar_indices = np.argsort(cossim)[::-1][1:6]

        print("Palabras más similares:")
        for sim_idx in most_similar_indices:
            similar_word = terms[sim_idx]
            similarity_score = cossim[sim_idx]
            print(f"- {similar_word} (similitud: {similarity_score:.4f})")
    else:
        print(f"La palabra '{word}' no está en el vocabulario.")


Palabra: computer
Palabras más similares:
- science (similitud: 0.1297)
- graphics (similitud: 0.1297)
- computers (similitud: 0.1250)
- software (similitud: 0.1148)
- university (similitud: 0.1071)
Palabra: science
Palabras más similares:
- scientific (similitud: 0.2408)
- scientists (similitud: 0.1961)
- computer (similitud: 0.1297)
- university (similitud: 0.1291)
- psychology (similitud: 0.1202)
Palabra: software
Palabras más similares:
- hardware (similitud: 0.1881)
- pc (similitud: 0.1438)
- packages (similitud: 0.1269)
- modem (similitud: 0.1164)
- disk (similitud: 0.1155)
La palabra 'system' no está en el vocabulario.
Palabra: data
Palabras más similares:
- transfer (similitud: 0.1544)
- encrypted (similitud: 0.1215)
- use (similitud: 0.1161)
- bus (similitud: 0.1149)
- rate (similitud: 0.1010)
